# 🤖 RAG AI-Bay - Building an AI-Powered Customer Care Service

## 🎯 Welcome to the Advanced RAG Challenge!

In this hands-on project, you'll solve a real-world problem: How can customers get instant, accurate answers to their questions about a second-hand marketplace? You'll build a **Retrieval-Augmented Generation (RAG)** system that does exactly that.
The dataset is very close to what we could have in reality!

---

---

## 📄 Step 2: Loading the Customer Care FAQ Data

We'll work with a JSON file containing frequently asked questions about the Olympics.

### 📊 Data Structure:

```json
[
  {
    "faq_id": "21561426636690",
    "faq_title": "**Title:** Professional...",
    "faq_body": "**The Steps for Sellin..",
    "updated_at": "2024-11-18T14:28",
  },
  ...
]
```

Let's load and explore the data!

In [59]:
# Load the FAQ data
from pathlib import Path
import json

# Get the notebook's directory
notebook_dir = Path.cwd()
data_path = notebook_dir / "data" / "faq_en.json"
with open(data_path, "r") as f:
    faq_data = json.load(f)

print(f"✅ Loaded {len(faq_data)} FAQ entries!\n")

✅ Loaded 327 FAQ entries!



In [60]:
# Display first entry
faq_data[0]

{'faq_id': 21561426636690,
 'faq_title': '**Title:** Professional Seller: How to Sell and Ship a Bundle?',
 'faq_body': '**The Steps for Selling in Lots**  \n**What to Do if One or More Items in the Lot Are Unavailable?**  \n**Can I Disable the Lot Purchase Option for My Profile?**\n\n---\n\n### The Steps for Selling in Lots\n✔ When a buyer purchases several of your items as a lot, you’ll be notified through the secure messaging system.  \n✔ Click on “Is the order available?”  \n✔ You can then view the contents of the lot created by the buyer and check whether all items are still available.  \n✔ Click on “Confirm availability.”  \n✔ The purchase is then validated, and you can proceed with shipping the package or arranging in-store pickup following the usual steps. See also:  \n**Pro Seller: How do I ship my package?**  \n**Pro Seller: Steps for a sale with secure payment and in-store pickup**\n\n---\n\n### What to Do if One or More Items in the Lot Are Unavailable?\nIf one or more item

In [62]:
# Display sample entries
print("📋 Sample FAQ Entries:\n")
for i, entry in enumerate(faq_data[:3], 1):
    print(f"{i}. {entry['faq_title']}")
    print(f"{entry['faq_body'][:100]}...\n")

print(f"💡 Total entries: {len(faq_data)}")

📋 Sample FAQ Entries:

1. **Title:** Professional Seller: How to Sell and Ship a Bundle?
**The Steps for Selling in Lots**  
**What to Do if One or More Items in the Lot Are Unavailable?** ...

2. **Title:** I’m Looking for a Listing on the AI-Bay.fr Website
You are currently on the **AI-Bay Help Center**.  
The search bar here only provides access to infor...

3. **Title:** The Data Protection Officer of AI-Bay
**What Is a Data Protection Officer (DPO)?**  
The Data Protection Officer (DPO) is the person respo...

💡 Total entries: 327


👀 These documents are significantly longer than the previous exercice, aren't they?

#### Question 1: In your opinion, what kind of preprocessing should we add in comparison with the previous exercice, and why ? (you could list at least 3 extra steps!)

Click [here](https://docs.langchain.com/oss/python/integrations/splitters) for a hint (if you have really no idea)

In [ ]:
## Your answer here:
## We should remove non necessary formating
## We should chunk!
## we should maybe rephrase
##

🎯 Exercise 1: Using what you learned in the previous exercise, build an ingestion pipeline that processes raw text all the way to the vector store. Keep in mind you'll need to add some extra steps (which are crucial!) to handle these longer documents effectively. They're not super long though, so don't worry! 😊

try those queries to self-evaluate your rag:
- query = "Why can't I find my ad ?"

- query="On the AI-Bay website, I don’t have the option to add an IBAN or a payment card.  \nI went to Settings → Payment Methods, but there’s no way to make any changes."

- query = "Hello,  \n\nThank you in advance for permanently DELETING my AI-Bay account, as it has been “blocked” and I can no longer use it.  \n\nKind regards."
        


In [ ]:
### GO GO GO ! Code here.

In [ ]:
# Clean text
import re


def remove_markdown_links_or_images(text: str):
    """
    Remove markdown links and images from the text
    - Images: ![alt text](url) -> removed completely
    - Links: [text](url) -> text only
    """
    # Remove images completely (they start with !)
    text = re.sub(r"!\[.*?\]\(.*?\)", "", text)
    # Convert links to just their text content
    text = re.sub(r"\[(.*?)\]\(.*?\)", r"\1", text)
    return text


def remove_asterisks(text: str):
    """
    Remove asterisks from the text
    """
    return re.sub(r"\*", "", text)


def clean_text(text: str):
    """Clean the text from markdown links and images, and remove asterisks."""
    text = remove_markdown_links_or_images(text)
    text = remove_asterisks(text)
    return text


for faq in faq_data:
    faq["faq_body"] = clean_text(faq["faq_body"])

In [ ]:
from langchain_core.documents import Document


documents = []
for faq in faq_data:
    documents.append(
        Document(
            page_content=faq["faq_body"],
            metadata={
                "faq_id": faq["faq_id"],
                "faq_body": faq["faq_body"],
                "faq_title": faq["faq_title"],
                "updated_at": faq["updated_at"],
            },
        )
    )

In [ ]:
from langchain_text_splitters.markdown import MarkdownTextSplitter

CHUNK_SIZE = 300
CHUNK_OVERLAP = 70
PAGE_CONTENT_WITH_TITLE_KEY = "page_content_with_title"
TITLE_KEY = "faq_title"


def approx_token_length(text: str) -> int:
    """Approximate the token length of a text."""
    return len(text) / 4


splitter = MarkdownTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=approx_token_length,
)

In [73]:
chunks = splitter.split_documents(documents)

In [80]:
print(len(chunks))

592


In [74]:
approx_token_length(chunks[0].page_content)

189.0

In [ ]:
chunks[0].page_content


def add_title_to_chunk(chunk: Document, title_key: str = TITLE_KEY):
    chunk.page_content = chunk.metadata[title_key] + "\n\n" + chunk.page_content


for chunk in chunks:
    add_title_to_chunk(chunk)

In [77]:
print(chunks[0].page_content)

**Title:** Professional Seller: How to Sell and Ship a Bundle?

The Steps for Selling in Lots  
What to Do if One or More Items in the Lot Are Unavailable?  
Can I Disable the Lot Purchase Option for My Profile?

---

### The Steps for Selling in Lots
✔ When a buyer purchases several of your items as a lot, you’ll be notified through the secure messaging system.  
✔ Click on “Is the order available?”  
✔ You can then view the contents of the lot created by the buyer and check whether all items are still available.  
✔ Click on “Confirm availability.”  
✔ The purchase is then validated, and you can proceed with shipping the package or arranging in-store pickup following the usual steps. See also:  
Pro Seller: How do I ship my package?  
Pro Seller: Steps for a sale with secure payment and in-store pickup

---


In [ ]:
from langchain_qdrant import QdrantVectorStore
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

# Initialize the embedding model

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)


vector_store = QdrantVectorStore.from_documents(
    chunks, embedding=embeddings, collection_name="faq_collection", location=":memory:"
)

In [ ]:
query = "Why can't I find my ad ?"
query = "Hello,  \n\nThank you in advance for permanently DELETING my AI-Bay account, as it has been “blocked” and I can no longer use it.  \n\nKind regards"
docs_and_scores = vector_store.similarity_search_with_relevance_scores(
    query, k=4, score_threshold=0.5
)

for doc, score in docs_and_scores:
    print(f"Score: {score}")
    print(f"Document: {doc.page_content}")
    print("\n")

Score: 0.8036824323383683
Document: **How Do I Delete My Account?**

To delete your AI-Bay account:  

From your computer  
1. Log in to your AI-Bay account.  
2. Go to Settings.  
3. At the bottom of the page, click “Delete my account.”  
4. Confirm the deletion.  

From the iOS app  
1. Log in to your AI-Bay account.  
2. Go to Settings → Personal Information.  
3. At the bottom of the page, click “Delete my account.”  
4. Confirm the deletion.  

Important: You cannot delete your account if a transaction is in progress. Please wait until all transactions are completed before proceeding.  

Personal data management after account deletion  
Deleting your account also removes your personal data from all our systems within a maximum of 30 days, with an average processing time of 10 days.  

Note: Account deletion is not yet available from the Android app.  
If you wish to delete your account, please do so from a computer.  
If you want to delete your professional account, please refer t

🎯 Exercise 2: now try to add a hybrid search to the pipeline. 
[hint here](https://docs.langchain.com/oss/python/integrations/vectorstores/qdrant)

Are the results better ? Why ?

In [ ]:
### GO GO GO ! Code here.

In [ ]:
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams

sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")


vector_store = QdrantVectorStore.from_documents(
    chunks,
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse",
    collection_name="faq_collection",
    location=":memory:",
)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
query = "Why can't I find my ad ?"
query = "Hello,  \n\nThank you in advance for permanently DELETING my AI-Bay account, as it has been “blocked” and I can no longer use it.  \n\nKind regards"
docs_and_scores = vector_store.similarity_search_with_relevance_scores(
    query, k=4, score_threshold=0.5
)

for doc, score in docs_and_scores:
    print(f"Score: {score}")
    print(f"Document: {doc.page_content}")
    print("\n")

Score: 0.7666666666666666
Document: **Title:** My account is blocked — what should I do?

To understand why your account has been blocked, we recommend reviewing our Posting Rules, Terms of Use, or Code of Conduct. We block accounts that do not comply with these conditions. You can contest the blocking of your account by sending a request to our customer service team.

Why can my account be blocked?  
To ensure the safety of all AI-Bay users, we may block your account if we detect suspicious behavior or activity that violates our Posting Rules, Terms of Use, or Code of Conduct.  
Here are some non-exhaustive examples:

- Inappropriate behavior: discriminatory attitude, insults, threats.  
- Fraud attempt: suspicious activity toward other users, scam attempts.  
- Prohibited content: sale of illegal products or services.

How can I contest the blocking of my account?  
If you believe your account was blocked by mistake, you can send an unblocking request to our team.

Important: Unblock

🎯 **Exercise 3**: Create a `RagConversation` class that handles:
- Document retrieval from your vector store
- Response generation using the LLM
- Conversation history management

See the message history implementation example below:

In [ ]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage

memory = []

messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Hello, how are you?"),
    AIMessage(content="I'm doing well, thank you!"),
]

memory.extend(messages)

for message in memory:
    # message.pretty_print()
    print(f"{message.type}: {message.content}\n")

system: You are a helpful assistant

human: Hello, how are you?

ai: I'm doing well, thank you!



In [ ]:
### GO GO GO ! Code here.

from langchain_core.messages.base import BaseMessage
from langchain.messages import HumanMessage, AIMessage, SystemMessage


class RagConversation:
    def __init__(self, vector_store, llm):
        self.vector_store = vector_store
        self.llm = llm
        self.history = []

    def add_message(self, message: BaseMessage):
        self.history.append(message)

    def history_to_string(self):
        return "\n".join(
            [f"{message.type}: {message.content}" for message in self.history]
        )

    def get_response(self, question):
        # TODO: code this function
        return

In [ ]:
def format_docs_alternative(docs):
    """
    Format retrieved documents for inclusion in prompt

    Args:
        docs: List of Document objects

    Returns:
        Formatted string with numbered documents
    """
    formatted = []

    for i, doc in enumerate(docs, 1):
        formatted.append(
            f"Document {i}:\n{doc.metadata['faq_body']}\nSource: {doc.metadata['faq_id']}"
        )

    return "\n\n".join(formatted)

In [ ]:
### GOGOOG !

In [96]:
# Define the prompt template
prompt_template = """You are a helpful assistant answering questions about customer care for AI-Bay.

Use the following context documents to answer the user's question. If the answer is not in the provided documents, say "I don't have that information in the provided documents."

Context Documents:
{context}

Here is the conversation history:
{history}

User Question: {question}

Instructions:
1. Answer based ONLY on the provided documents
2. Be specific and cite which document(s) you used
3. If information is unclear or missing, say so
4. Keep answers concise but complete
5. Use a friendly, informative tone
Answer:"""

In [ ]:
from langchain_core.messages.base import BaseMessage
from langchain.messages import HumanMessage, AIMessage, SystemMessage


class RagConversation:
    def __init__(self, vector_store, llm, history=None):
        self.vector_store = vector_store
        self.llm = llm
        self.history = history if history else []

    def add_message(self, message: BaseMessage):
        self.history.append(message)

    def history_to_string(self):
        return "\n".join(
            [f"{message.type}: {message.content}" for message in self.history]
        )

    def get_response(self, question):
        # TODO: code this function
        context = self.vector_store.similarity_search(question, k=4)
        context_str = format_docs_alternative(context)
        prompt = prompt_template.format(
            context=context_str, history=self.history_to_string(), question=question
        )
        response = self.llm.invoke(prompt)
        self.history.append(HumanMessage(content=question))
        self.history.append(AIMessage(content=response.content))
        return response

In [109]:
MODEL_NAME = "gpt-5-nano"
REASONING_EFFORT = "minimal"  # could be   "minimal" | "low" | "medium" | "high"  see [https://platform.openai.com/docs/guides/latest-model]
TEMPERATURE = 0

# Initialize the LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=MODEL_NAME, temperature=TEMPERATURE, reasoning_effort=REASONING_EFFORT
)

rag_conversation = RagConversation(vector_store, llm)
rag_conversation.get_response("Hello, how are you?")
rag_conversation.history

[HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi there! I’m doing well, thanks for asking. How can I help you with AI-Bay today? (If you were asking about contacting a seller, posting an ad, or messaging, I can guide you using the steps in the documents.)\n\nUsed documents: Document 1 and Document 4 for contact/messaging options; Document 2 for viewing messages across devices; Document 3 for posting ads.', additional_kwargs={}, response_metadata={})]

Use it with the gradio UI !

In [ ]:
import gradio as gr

K = 5

rag_conversation = RagConversation(vector_store, llm)


def rag_assistant_response(message, history):
    return rag_conversation.get_response(message).content


gr.ChatInterface(fn=rag_assistant_response).launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


---

## 🎉 Congratulations!


---

## 🚀 Next Steps:

### 🔧 Immediate Improvements:
1. Add more FAQ entries to the database
2. Experiment with different chunk sizes
3. Try different embedding models
4. Adjust retrieval count (k parameter)
5. Fine-tune the prompt template

### 🎓 Advanced Topics to Explore:
1. **Hybrid Search:** Combine keyword + semantic
2. **Reranking:** Score and reorder retrieved docs
3. **Query Expansion:** Generate multiple query variations
4. **Metadata Filtering:** Filter by date, source, etc.
5. **Multi-modal RAG:** Include images, tables
6. **Streaming Responses:** Real-time token generation
7. **Conversation Memory:** Multi-turn conversations
8. **Evaluation Metrics:** Measure RAG quality

### 🌟 Project Ideas:
1. Build a RAG system for your company docs
2. Create a research paper Q&A system
3. Make a personal knowledge base assistant
4. Build a code documentation helper
5. Create a study buddy for textbooks

---

## 💭 Final Thoughts

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 15px; margin: 20px 0;">
    <h3 style="margin-top: 0;">🎓 Remember:</h3>
    <p><strong>"The best RAG system is the one that solves your specific problem."</strong></p>
    <ul>
        <li>Start simple and iterate</li>
        <li>Measure what matters to your users</li>
        <li>Focus on retrieval quality first</li>
        <li>Then optimize generation</li>
        <li>Always test with real users</li>
    </ul>
</div>

---

<div style="text-align: center; background: linear-gradient(135deg, #FF6B6B 0%, #FFE66D 100%); padding: 30px; border-radius: 15px; margin: 30px 0;">
    <h2 style="margin: 0; font-size: 2.5em;">🏅 You're Now a RAG Expert!</h2>
    <p style="margin: 20px 0 0 0; font-size: 1.3em; font-weight: bold;">Go forth and build amazing AI-powered systems! 🚀</p>
</div>

---

### 📚 Additional Resources:

- [LangChain Documentation](https://python.langchain.com/)
- [RAG Best Practices](https://www.anthropic.com/research)

### 🤝 Community:

- LangChain Discord
- r/MachineLearning
- AI Stack Exchange
- Hugging Face Forums

---

**Happy Building! 🎉**